In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWhuRf7TqBJPb-ESGSP9owIXlFiEvstzlneuZOecRyBBwE1rH1BpwhQ

Successfully saved authorization token.


In [3]:

# Import the MODIS land cover collection.
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# Import the MODIS land surface temperature collection.
lst = ee.ImageCollection('MODIS/006/MOD11A1')

# Import the USGS ground elevation image.
elv = ee.Image('USGS/SRTMGL1_003')

# Import JRC Global Surface Water
water = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")

# Wetness
wetness = ee.ImageCollection('Oxford/MAP/TCW_5km_Monthly')
wetness_img = wetness.mean()
wetness_img = wetness_img.select('Mean')
# Import the MODIS EVI
evi = ee.ImageCollection('MODIS/MOD09GA_006_EVI')
evi_img = evi.mean()

# Initial date of interest (inclusive).
i_date = '2017-01-01'

# Final date of interest (exclusive).
f_date = '2020-01-01'

# Selection of appropriate bands and dates for LST.
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(i_date, f_date)

# Reduce the LST collection by mean.
lst_img = lst.mean()

# Adjust for scale factor.
lst_img = lst_img.select('LST_Day_1km').multiply(0.02)

# Convert Kelvin to Celsius.
lst_img = lst_img.select('LST_Day_1km').add(-273.15)

# Create a geodesic polygon.
xmin, xmax, ymin, ymax = -87.93044, -85.23703, 31.82364, 35.0167

polygon = ee.Geometry.Polygon(coords = [[[xmin,ymin], [xmin,ymax ], [xmax,ymin], [xmax,ymax ]]])


In [4]:

from IPython.display import Image

# Create a URL to the styled image for a region around France.
url = lst_img.getThumbUrl({
    'min': 10, 'max': 30, 'dimensions': 512, 'region': polygon,
    'palette': ['blue', 'yellow', 'orange', 'red']})
print(url)

# Display the thumbnail land surface temperature in France.
print('\nPlease wait while the thumbnail loads, it may take a moment...')
Image(url=url)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/2df16ac3a7068bf8e56c0155bccf907a-e285100205332b1b74e24efb123a973c:getPixels

Please wait while the thumbnail loads, it may take a moment...


In [5]:

# Make pixels with elevation below sea level transparent.
elv_img = elv.updateMask(elv.gt(0))

# Display the thumbnail of styled elevation in France.
Image(url=elv_img.getThumbURL({
    'min': 0, 'max': 2000, 'dimensions': 512, 'region': polygon,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))


In [6]:
Image(url=wetness_img.getThumbURL({
    'min': -0.6, 'max': 0.5,  'region': polygon,
    'palette': ['ffffff', 'fcd163', '99b718', '66a000', '3e8601', '207401', '056201','004c00', '011301']}))

In [52]:
task_wetness = ee.batch.Export.image.toDrive(image=wetness_img,
                                     description='wetness_gallopavo_1000',
                                     scale=1000,
                                     region=polygon,
                                     fileNamePrefix='wetness_gallopavo_1000',
                                     crs='EPSG:32616',
                                     fileFormat='GeoTIFF')
task_wetness.start()

In [53]:
task_wetness.status()

{'state': 'READY',
 'description': 'wetness_gallopavo_1000',
 'creation_timestamp_ms': 1654385009164,
 'update_timestamp_ms': 1654385009164,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'LQMWLY5P3SA4Z5WBDVIZEXZA',
 'name': 'projects/earthengine-legacy/operations/LQMWLY5P3SA4Z5WBDVIZEXZA'}

In [55]:
task_evi = ee.batch.Export.image.toDrive(image=evi_img,
                                     description='evi_gallopavo_1000',
                                     scale=1000,
                                     region=polygon,
                                     fileNamePrefix='evi_gallopavo_1000',
                                     crs='EPSG:32616',
                                     fileFormat='GeoTIFF')
task_evi.start()


In [56]:
task_evi.status()


{'state': 'READY',
 'description': 'evi_gallopavo_1000',
 'creation_timestamp_ms': 1654385032077,
 'update_timestamp_ms': 1654385032077,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'G3Z467YB2VYU3J7WWGQ4HXML',
 'name': 'projects/earthengine-legacy/operations/G3Z467YB2VYU3J7WWGQ4HXML'}

In [57]:
task_lst = ee.batch.Export.image.toDrive(image=lst_img,
                                     description='lst_gallopavo_1000',
                                     scale=1000,
                                     region=polygon,
                                     fileNamePrefix='lst_gallopavo_1000',
                                     crs='EPSG:32616',
                                     fileFormat='GeoTIFF')
task_lst.start()


In [58]:
task_lst.status()

{'state': 'READY',
 'description': 'lst_gallopavo_1000',
 'creation_timestamp_ms': 1654385044401,
 'update_timestamp_ms': 1654385044401,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'QFJZC52PCW222AE2R5MWMYCG',
 'name': 'projects/earthengine-legacy/operations/QFJZC52PCW222AE2R5MWMYCG'}

In [59]:
task_elv = ee.batch.Export.image.toDrive(image=elv_img,
                                     description='elv_gallopavo_1000',
                                     scale=1000,
                                     region=polygon,
                                     fileNamePrefix='elv_gallopavo_1000',
                                     crs='EPSG:32616',
                                     fileFormat='GeoTIFF')
task_elv.start()


In [60]:
task_elv.status()

{'state': 'READY',
 'description': 'elv_gallopavo_1000',
 'creation_timestamp_ms': 1654385059700,
 'update_timestamp_ms': 1654385059700,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'K25QJF3SU7LUEIWSUKSQGYIX',
 'name': 'projects/earthengine-legacy/operations/K25QJF3SU7LUEIWSUKSQGYIX'}